# Purity Network
Quantum Neural Network (QNN) that compute the purity of a given state

In [1]:
import os
from matplotlib import pyplot as plt
%matplotlib inline
from IPython.display import clear_output

import pennylane as qml
from pennylane import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
np.random.seed(42)

## Constants

### Dataset

In [4]:
data_folder = "data"

n_qumodes_rho = 1
n_qumodes_psi = n_qumodes_rho * 2 # for purification
n_qumodes_pn = n_qumodes_rho * 4 # input: [ρ, ρ]
n_qumodes_total = n_qumodes_psi * 2

cutoff = 3
size_hilbert = cutoff**n_qumodes_rho

ratio_train = 0.75

### State network

In [5]:
n_layers_sp = 20

### Purity Network

In [6]:
n_layers_pn = 2
batch_size_pn = 16

passive_std = 0.1
active_std = 0.001

## Loading the dataset

### Loading the files

In [7]:
rhos = np.load(os.path.join(data_folder, "rhos.npy"))
purities = np.load(os.path.join(data_folder, "purities.npy"))
list_params = np.load(os.path.join(data_folder, "list_params.npy"))

In [8]:
n_samples = len(rhos)
n_samples_train = int(ratio_train*n_samples)
n_samples_test = n_samples - n_samples_train

In [9]:
samples_idx = np.random.choice(n_samples, size=n_samples, replace=False)
rhos = rhos[samples_idx]
purities = purities[samples_idx]
list_params = list_params[samples_idx]

### Dividing train and test

In [10]:
X_train = list_params[:n_samples_train]
X_test = list_params[n_samples_train:]

Y_train = purities[:n_samples_train]
Y_test = purities[n_samples_train:]

## State Preparation Network

In [11]:
def Interferometer(theta, phi, rphi, q):
	# parameterised interferometer acting on N qumodes
    # theta is a list of length N(N-1)/2
    # phi is a list of length N(N-1)/2
    # rphi is a list of length N-1
	# q is the list of qumodes the interferometer is to be applied to
    N = len(q)

    if N == 1:
        # the interferometer is a single rotation
        qml.Rotation(rphi[0], wires=q[0])
        return

    n = 0 # keep track of free parameters

    # Apply the Clements beamsplitter array
    # The array depth is N
    for l in range(N):
        for k, (q1, q2) in enumerate(zip(q[:-1], q[1:])):
            #skip even or odd pairs depending on layer
            if (l+k)%2 != 1:
                qml.Beamsplitter(theta[n], phi[n], wires=(q1, q2))
                n += 1

    # apply the final local phase shifts to all modes except the last one
    for i in range(len(q)-1):
        qml.Rotation(rphi[i], wires=q[i])

In [12]:
def layer(i, q, params):
    sq_r, sq_phi, d_r, d_phi, inter_theta, inter_phi, inter_rphi, kappa = tuple(params)

    Interferometer(inter_theta[2*i], inter_phi[2*i], inter_rphi[2*i], q)
    
    for j in range(len(q)):
        qml.Squeezing(sq_r[i,j], sq_phi[i,j], wires=q[j])
        
    Interferometer(inter_theta[2*i+1], inter_phi[2*i+1], inter_rphi[2*i+1], q)
    
    for j in range(len(q)):
        qml.Displacement(d_r[i,j], d_phi[i,j], wires=q[j])
        
    for j in range(len(q)):
        qml.Kerr(kappa[i,j], wires=q[j])

    return q

In [13]:
def state_preparation_network(q, n_layers, parameters):
    for i in range(n_layers):
        layer(i, q, parameters)

## Purity Network

### Parameters

In [14]:
# Squeeze gate
pn_sq_r = np.random.normal(size=[n_layers_pn, n_qumodes_pn], scale=active_std)
pn_sq_phi = np.random.normal(size=[n_layers_pn, n_qumodes_pn], scale=passive_std)

# Displacement gate
pn_d_r = np.random.normal(size=[n_layers_pn, n_qumodes_pn], scale=active_std)
pn_d_phi = np.random.normal(size=[n_layers_pn, n_qumodes_pn], scale=passive_std)

# Interferometer
pn_inter_theta = np.random.normal(size=[n_layers_pn*2, int(n_qumodes_pn*(n_qumodes_pn-1)/2)], scale=passive_std)
pn_inter_phi = np.random.normal(size=[n_layers_pn*2, int(n_qumodes_pn*(n_qumodes_pn-1)/2)], scale=passive_std)
pn_inter_rphi = np.random.normal(size=[n_layers_pn*2, n_qumodes_pn-1], scale=passive_std)

# Kerr gate
pn_kappa = np.random.normal(size=[n_layers_pn, n_qumodes_pn*2], scale=active_std)

In [15]:
pn_params = [pn_sq_r, pn_sq_phi, pn_d_r, pn_d_phi, pn_inter_theta, pn_inter_phi, pn_inter_rphi, pn_kappa]

### Architecture

In [16]:
def purification_network(q, n_layers, params):
    # First layers with same size
    n_qumodes = len(q)

    for i in range(n_layers - (n_qumodes-2)): # same-size layers
        layer(i, q, params)
        
    # Progressive size reduction for the last layers
    for i in range(n_qumodes-2):
        l = i+(n_layers - (n_qumodes-2))
        layer(l, q[i+1:], params)

## Total Network

### Prepare the network

In [17]:
device = qml.device('strawberryfields.fock', wires=n_qumodes_total, cutoff_dim=cutoff)
q = range(n_qumodes_total)

In [18]:
@qml.qnode(device)
def qnetwork(pn_sq_r, pn_sq_phi, pn_d_r, pn_d_phi, pn_inter_theta, pn_inter_phi, pn_inter_rphi, pn_kappa,
             sp_sq_r, sp_sq_phi, sp_d_r, sp_d_phi, sp_inter_theta, sp_inter_phi, sp_inter_rphi, sp_kappa):
    pn_params = [pn_sq_r, pn_sq_phi, pn_d_r, pn_d_phi, pn_inter_theta, pn_inter_phi, pn_inter_rphi, pn_kappa]
    sp_params = [sp_sq_r, sp_sq_phi, sp_d_r, sp_d_phi, sp_inter_theta, sp_inter_phi, sp_inter_rphi, sp_kappa]
    
    state_preparation_network(q[:n_qumodes_psi], n_layers_sp, sp_params)
    state_preparation_network(q[n_qumodes_psi:], n_layers_sp, sp_params)
    purification_network(q[:n_qumodes_pn], n_layers_pn, pn_params)
    
    return qml.expval.X(q[-1])

In [19]:
def total_network(pn_params, sp_params):
    pn_sq_r, pn_sq_phi, pn_d_r, pn_d_phi, pn_inter_theta, pn_inter_phi, pn_inter_rphi, pn_kappa = pn_params
    sp_sq_r, sp_sq_phi, sp_d_r, sp_d_phi, sp_inter_theta, sp_inter_phi, sp_inter_rphi, sp_kappa = sp_params
    
    return qnetwork(pn_sq_r, pn_sq_phi, pn_d_r, pn_d_phi, pn_inter_theta, pn_inter_phi, pn_inter_rphi, pn_kappa,
                    sp_sq_r, sp_sq_phi, sp_d_r, sp_d_phi, sp_inter_theta, sp_inter_phi, sp_inter_rphi, sp_kappa)

### Cost and optimizer

In [20]:
def purity_mse(purity1, purity2):
    return np.mean(np.square(purity1 - purity2))

In [21]:
def cost(w, X_batch, Y_batch):
    print("output: ", total_network(w, X_batch[0]))
    Y_predict = []
    for i in range(len(X_batch)):
        Y_predict.append(total_network(w, X_batch[i]))
    Y_predict = np.array(Y_predict)
    print("mse: ", purity_mse(Y_predict, Y_batch))
    return purity_mse(Y_predict, Y_batch)

In [22]:
op = qml.AdamOptimizer(stepsize=50e-4)

### Training

In [23]:
cost_train_list = []
cost_test_list = []
i = 0

In [25]:
nb_epochs = 30000
batch_size_pn = 32
n_iters_test = 10

op.stepsize = 100e-4

for i in range(i, nb_epochs+i):
    cost_train = []
    cost_test = [] 
    samples_idx = np.random.choice(n_samples_train, size=n_samples_train, replace=False)
    
    for start_batch in range(0, n_samples_train, batch_size_pn):
        end_batch = min(start_batch+batch_size_pn, n_samples_train)
        idx_batch = samples_idx[start_batch:end_batch]

        print("start opti step")
        pn_params = op.step(lambda w: cost(w, X_train[idx_batch], Y_train[idx_batch]), pn_params)
        print("end opti step")
        cost_train_list.append(cost(pn_params, X_train[idx_batch], Y_train[idx_batch]))
        
    if i % n_iters_test == 0:
        samples_idx = np.random.choice(n_samples_test, size=n_samples_test, replace=False)
        cost_test_list.append(cost(pn_params, X_test[samples_idx], Y_test[samples_idx]))
            
    clear_output(wait=True)
    print('Cost train after step {:5d}: {: .7f}'.format(i, cost_train_list[-1]))
    print('Cost test after step {:5d}: {: .7f}'.format(i, cost_test_list[-1]))
i = i+1

start opti step
output:  Autograd ArrayBox with value 0.01977858581319233
mse:  Autograd ArrayBox with value 0.5359193824291212


KeyboardInterrupt: 

# Visualization and result

In [ ]:
start = 5
plt.rcParams['figure.figsize'] = (16,5)

plt.subplot(1,2,1)
plt.plot(range(i)[start:], cost_train_list[start:], 
         label="Training learning curve")
plt.legend()

plt.subplot(1,2,2)
plt.plot(range(0,i,n_iters_test)[start//n_iters_test:], cost_test_list[start//n_iters_test:], 
         label="Testing learning curve")
plt.legend()

In [ ]:
plt.subplot(1,2,1)
purity_pred = []
for j in range(n_samples_train):
    feed_dict = get_feed_dict(X_train[j])
    purity_pred.append(sess.run(purity_output, feed_dict=feed_dict))
#     df = df.append([{"Prediction": purity_pred, "Truth": Y_train[j]}])
# df.sort_values("Truth")
plt.scatter(Y_train, purity_pred, label="Train")
plt.plot([0,1],[0,1])
plt.xlabel("Truth")
plt.ylabel("Prediction")
plt.legend()

plt.subplot(1,2,2)
purity_pred = []
for j in range(n_samples_test):
    feed_dict = get_feed_dict(X_test[j])
    purity_pred.append(sess.run(purity_output, feed_dict=feed_dict))
#     df = df.append([{"Prediction": purity_pred, "Truth": Y_train[j]}])
# df.sort_values("Truth")
plt.scatter(Y_test, purity_pred, label="Test")
plt.plot([0,1],[0,1])
plt.xlabel("Truth")
plt.ylabel("Prediction")
plt.legend()

## Debug